In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prave\\Desktop\\Projects\\text_summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\prave\\Desktop\\Projects\\text_summarizer'

## WorkFlow for the given project
1. Update config.yaml
2. Update params.yaml
3. update entity
4. Update the configuration manager in src config
5. Update the componenets
6. Update the pipeline
7. Update the main.py
8. Update the app.py

In [5]:
# 1. config.yaml updated
# 2. params.yaml updated
#3 3. Updating Entity


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
# Updating the configuration manager in src config

from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config (self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])


        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config


In [7]:
# Updating Components
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

c:\Users\prave\anaconda3\envs\textsum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-07-22 23:00:19,286: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-22 23:00:19,294: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-22 23:00:19,294: INFO: common: created directory at: artifacts]
[2023-07-22 23:00:19,294: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\prave\anaconda3\envs\textsum\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 20%|█▉        | 10/51 [13:53<49:30, 72.45s/it]  

{'loss': 3.2995, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [24:36<33:23, 64.62s/it]

{'loss': 3.034, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 3.0745, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [57:15<01:07, 67.62s/it]

{'loss': 2.7473, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [58:23<00:00, 68.69s/it]


{'train_runtime': 3503.7687, 'train_samples_per_second': 0.234, 'train_steps_per_second': 0.015, 'train_loss': 3.0354243026060215, 'epoch': 1.0}
